In [1]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector
import numpy as np
import random
from tensorflow.python.keras import layers

In [2]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py

--2024-01-08 21:37:21--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2161 (2.1K) [text/plain]
Saving to: ‘roman_numerals.py’

roman_numerals.py   100%[===================>]   2.11K  --.-KB/s    in 0s      

2024-01-08 21:37:21 (18.3 MB/s) - ‘roman_numerals.py’ saved [2161/2161]



In [3]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py

--2024-01-08 21:37:26--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1812 (1.8K) [text/plain]
Saving to: ‘encoder.py’

encoder.py          100%[===================>]   1.77K  --.-KB/s    in 0s      

2024-01-08 21:37:27 (28.9 MB/s) - ‘encoder.py’ saved [1812/1812]



In [4]:
from encoder import CharacterTable
import roman_numerals as cnv

# Parameters for the model and dataset.
DATASET_SIZE = 200
INPUT = 14
OUTPUT = 4
# object to encode roman numbers to one-hot
romans = 'MDCLXVI '
rtable = CharacterTable(romans)

# object to encode arabic numbers to one-hot
chars = '0123456789 '
dtable = CharacterTable(chars)

In [5]:
seq_samples = []
seq_labels = []
used = []
repetitions = 0
generated = 0
print('Generating data...')
while len(seq_samples) < DATASET_SIZE:
    number = random.randint(1,200)
    # skip if already in the dataset
    if number in used: continue
    used.append(number)
    # roman input
    roman = cnv.convert(number)
    roman += ' ' * (INPUT - len(roman))

    # arabic output
    arabic = number

    seq_samples.append(arabic)
    seq_labels.append(roman)
print('Total roman numbers:', len(seq_samples))

Generating data...
Total roman numbers: 200


In [6]:
# one-hot encoding of all romans and arabic numbers
print('Vectorization...')
labels = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)

for i, sentence in enumerate(seq_labels):
    labels[i] = rtable.encode(sentence, INPUT)

print("labels",labels.shape)

Vectorization...
labels (200, 14, 8)


<ipython-input-6-3870f8e6f5b7>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)


In [7]:
samples = np.array(seq_samples)
samples.shape

(200,)

In [8]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [9]:
OUTPUT_SEQUENCE_LEN=14

model = Sequential()
model.add(Dense(16, input_dim=1) )
model.add(RepeatVector(OUTPUT_SEQUENCE_LEN)) #length of the text
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(8,activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy','mae'])
num_epochs = 0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                32        
                                                                 
 repeat_vector (RepeatVecto  (None, 14, 16)            0         
 r)                                                              
                                                                 
 lstm (LSTM)                 (None, 14, 128)           74240     
                                                                 
 lstm_1 (LSTM)               (None, 14, 128)           131584    
                                                                 
 lstm_2 (LSTM)               (None, 14, 128)           131584    
                                                                 
 dense_1 (Dense)             (None, 14, 8)             1032      
                                                        

In [10]:
def avg(list):
    return sum(list)/len(list)

EPOCHS = 50
# Train the model each generation and show predictions against the validation dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration*EPOCHS)
    H = model.fit(trainSamples, trainLabels, batch_size=128, epochs=EPOCHS,
                  validation_data=(testSamples, testLabels), verbose=0)
    print("Validation accuracy: {}".format(avg(H.history["val_accuracy"])))
    # Select 10 samples from the validation set at random to visualize errors.
    accuracy = 0

    for i in range(len(testSamples)):

        pred = model.predict(testSamples)
        roman = rtable.decode(testLabels[i])
        correct = roman
        number = testSamples[i]
        prediction = rtable.decode(pred[i])

        correctTxt = ""
        if correct == prediction:
            correctTxt+='OK!'
            accuracy+=1

        v = random.random()
        if v>0.9:
            print('expression:',number , end=' ')
            print('correct:', roman, end=' ')
            print('predicted:', prediction, end=' ')
            print(correctTxt)

    print('accuracy=',accuracy,"/",len(testSamples),"=",accuracy/len(testSamples))

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 27ms/step
expression: 1 correct: I              predicted: VI   VVVVMMMMM 
4/4 [==============================] - 0s 28ms/step
expression: 18 correct: XVIII          predicted: XIX            
4/4 [==============================] - 0s 26ms/step
expression: 16 correct: XVI            predicted: XV             
4/4 [==============================] - 0s 35ms/step
expression: 44 correct: XLIV           predicted: XLIII          
4/4 [==============================] - 0s 47ms/step
accuracy= 0 / 100 = 0.0

--------------------------------------------------
Iteration 7850
Validation accuracy: 0.8509857141971588
4/4 [==============================] - 0s 26ms/step
expression: 179 correct: CLXXIX         predicted: CLXXXII        
4/4 [==============================] - 0s 35ms/step
expression: 157 correct: CLVII          predicted: CLV            
4/4 [==============================] - 0s 25ms/step
expre